# Tools and Routing

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain.agents import tool

In [3]:
@tool # converts the function into a langchain tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

search.name, search.description, search.args

('search',
 'Search for weather online',
 {'query': {'title': 'Query', 'type': 'string'}})

In [4]:
# Define a structure for the input schema
# Important since the LLM use it as a prompt
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

search.name, search.description, search.args

('search',
 'Search for weather online',
 {'query': {'description': 'Thing to search for',
   'title': 'Query',
   'type': 'string'}})

In [5]:
search.run("CDG")

'42f'

## Application: get temperature given a location latitude and longitude

In [6]:
import requests
from datetime import datetime

# Input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(description="Latitude of the location to fetch weather data")
    longitude: float = Field(description="Longitude of the location to fetch weather data")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # Parameters for the request
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "forecast_days": 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.now()
    time_list = [datetime.fromisoformat(time_str.replace("Z", "+00:00")) for time_str in results["hourly"]["time"]]
    temperature_list = results["hourly"]["temperature_2m"]
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f"{current_temperature}°C"

In [7]:
get_current_temperature.name, get_current_temperature.description, get_current_temperature.args

('get_current_temperature',
 'Fetch current temperature for given coordinates.',
 {'latitude': {'description': 'Latitude of the location to fetch weather data',
   'title': 'Latitude',
   'type': 'number'},
  'longitude': {'description': 'Longitude of the location to fetch weather data',
   'title': 'Longitude',
   'type': 'number'}})

In [8]:
# Convert the tool to an actual OpenAI function
from langchain_core.utils.function_calling import convert_to_openai_function
get_current_temperature_openai = convert_to_openai_function(get_current_temperature)

In [ ]:
from langchain_openai import ChatOpenAI
import json

# Create the request and ask the model to identify the function inputs
latitude = 3.87
longitude = 11.52

ask_weather = """What's the current temperature in:
latitude = {latitude}
longitude = {longitude}
"""

messages = [
    {"role": "user", "content": ask_weather.format(latitude=latitude, longitude=longitude)},
]

model = ChatOpenAI().bind(functions=[get_current_temperature_openai])
response = model.invoke(messages)
messages.append(response)

# Retrieve the inputs and execute the function
args = json.loads(response.additional_kwargs["function_call"]["arguments"])
temperature = get_current_temperature.run(args)

# Add the function output to the conversation and display the final response
messages.append({
    "role": "function",
    "name": response.additional_kwargs["function_call"]["name"],
    "content": temperature,
})

response = model.invoke(messages)
response.content

'The current temperature at latitude 3.87 and longitude 11.52 is 27.8°C.'